<a href="https://colab.research.google.com/github/Dansah2/Identifying-Age-Related-Conditions/blob/main/GB_Tree_Model_ICR_Identifying_Age_Related_Conditions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ICR - Identifying Age-Related Conditions

Kaggle Dataset Download API Command:

kaggle competitions download -c icr-identify-age-related-conditions

Predict whether a subject has or has not been diagnosed with one of these conditions -- a binary classification problem.

##Project Outline:

1) Download the dataset

2) Explore/Analyze the data

3) Preprocess and organize the data

4) Create and Train baseline Model

5) Save the Model

## Download the Dataset

1) Install required libraries

2) Import required libraries

3) Obtain the preprocessed data previously saved to Google Drive


#### Install Required Libraries

In [ ]:
!pip install -q -U keras-tuner
!pip install -q -U scikit-learn
!pip install -q -U numpy
!pip install -q -U tensorflow_decision_forests
!pip install -q -U bentoml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 40.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.8 MB/s eta 0:0

#### Import Required Libraries

In [ ]:
# loading and handeling data
import numpy as np
import pandas as pd

# model training
import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf

# downloading data
from google.colab import drive

# Training the model with k-fold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# hyperparameter tuning
import keras_tuner as kt

# reload model using bentoml
from pathlib import Path
import bentoml

#### Obtain the preprocessed data previously saved to Google Drive


In [ ]:
# Mount google drive to store Kaggle API for future use
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create a function to read the data into a dataframe

def read_function(csv_file):

    return pd.read_csv(csv_file)

clean_train = read_function('/content/drive/My Drive/ICR_Project/train_df.csv')

## Create and Train baseline model
1) Calculate the weights

2) Create the model / callbacks

3) Define the plot function

4) Train the model

###Calculate the weights

In [ ]:
def get_weights(train_df, target):
  # Calculate the number of samples for each label.
  neg, pos = np.bincount(train_df[target])

  # Calculate total samples.
  total = neg + pos

  # Calculate the weight for each label.
  weight_for_0 = (1 / neg) * (total / 2.0)
  weight_for_1 = (1 / pos) * (total / 2.0)

  class_weight = {0: weight_for_0, 1: weight_for_1}

  print(f'Weight for class 0: {weight_for_0:.2f}')
  print(f'Weight for class 1: {weight_for_1:.2f}')

  return class_weight

class_weight = get_weights(clean_train, 'Class')

Weight for class 0: 0.61
Weight for class 1: 2.86


###Select the model

In [ ]:
#Look at the models to select from
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [ ]:
# check config options
model = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")

Resolve hyper-parameter template "benchmark_rank1" to "benchmark_rank1@v1" -> {'growing_strategy': 'BEST_FIRST_GLOBAL', 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}.
Use /tmp/tmp4c1clar5 as temporary training directory


###Tune hyperparameters


In [ ]:
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(clean_train, label="Class")

tuner = tfdf.tuner.RandomSearch(num_trials=20)

# Hyper-parameters to optimize.
tuner.choice("max_depth", [3, 4, 5, 6, 7, 8, 9, 10])
tuner.choice("num_trees", [400, 500, 600, 700, 800])

model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
model.fit(tf_dataset)

print(model.summary())

Use /tmp/tmpoyp0cyck as temporary training directory
Reading training dataset...
Training dataset read in 0:00:06.875294. Found 617 examples.
Training model...
Model trained in 0:00:08.770881
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (55):
	AB
	AF
	AH
	AM
	AR
	AX
	AY
	AZ
	BC
	BD_
	BN
	BP
	BQ
	BR
	BZ
	CB
	CC
	CD_
	CF
	CH
	CL
	CR
	CS
	CU
	CW_
	DA
	DE
	DF
	DH
	DI
	DL
	DN
	DU
	DV
	DY
	EB
	EE
	EG
	EH
	EL
	EP
	EU
	FC
	FD_
	FE
	FI
	FL
	FR
	FS
	GB
	GE
	GF
	GH
	GI
	GL

No weights

Variable Importance: INV_MEAN_MIN_D

###Train the model

In [ ]:
# Create list of ids for the creation of oof dataframe.
ID_LIST = clean_train.index

# Create a dataframe of required size with zero values.
oof = pd.DataFrame(data=np.zeros((len(ID_LIST),1)), index=ID_LIST)

# Create an empty dictionary to store the models trained for each fold.
models = {}

# Create empty dict to save metircs for the models trained for each fold.
accuracy = {}
cross_entropy = {}

# Save the name of the label column to a variable.
label = "Class"

# Creates a GroupKFold with 5 splits
kf = KFold(n_splits=5)

In [ ]:
def train_model(dataset_df, class_weight):
  # Loop through each fold
  for i, (train_index, valid_index) in enumerate(kf.split(X=dataset_df)):
    print('##### Fold',i+1)

    # Fetch values corresponding to the index
    train_df = dataset_df.iloc[train_index]
    valid_df = dataset_df.iloc[valid_index]
    valid_ids = valid_df.index.values

    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=label)
    valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df, label=label)

    # Define the model and metrics
    model = tfdf.keras.GradientBoostedTreesModel(max_depth=3, num_trees=700)
    model.compile(metrics=["accuracy", "binary_crossentropy"])

    # Train the model
    model.fit(x=train_ds, class_weight=class_weight)

    # Store the model
    models[f"fold_{i+1}"] = model

    # Predict OOF value for validation data
    predict = model.predict(x=valid_ds)

    # Store the predictions in oof dataframe
    oof.loc[valid_ids, 0] = predict.flatten()

    # Evaluate and store the metrics in respective dicts
    evaluation = model.evaluate(x=valid_ds,return_dict=True)
    accuracy[f"fold_{i+1}"] = evaluation["accuracy"]
    cross_entropy[f"fold_{i+1}"]= evaluation["binary_crossentropy"]

  average_loss = 0
  average_acc = 0

  for _model in  models:
      average_loss += cross_entropy[_model]
      average_acc += accuracy[_model]
      print(f"\n{_model}: acc: {accuracy[_model]:.4f} loss: {cross_entropy[_model]:.4f}")

  print(f"\nAverage accuracy: {average_acc/5:.4f}  Average loss: {average_loss/5:.4f}")

  return model

model = train_model(clean_train, class_weight)

##### Fold 1


Use /tmp/tmp4gp4zvpb as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.510059. Found 493 examples.
Training model...
Model trained in 0:00:00.742427
Compiling model...
Model compiled.
1/1 [==============================] - 1s 762ms/step - loss: 0.0000e+00 - accuracy: 0.9597 - binary_crossentropy: 0.1458
##### Fold 2


Use /tmp/tmpyzoumg97 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.226315. Found 493 examples.
Training model...
Model trained in 0:00:00.444301
Compiling model...
Model compiled.


1/1 [==============================] - 0s 300ms/step - loss: 0.0000e+00 - accuracy: 0.8790 - binary_crossentropy: 0.2557
##### Fold 3


Use /tmp/tmp33omkhbd as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.753258. Found 494 examples.
Training model...
Model trained in 0:00:00.630651
Compiling model...


...

1/1 [==============================] - 0s 303ms/step - loss: 0.0000e+00 - accuracy: 0.9187 - binary_crossentropy: 0.2841
##### Fold 5


Use /tmp/tmp6pzkc80r as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.759190. Found 494 examples.
Training model...
Model trained in 0:00:00.534991
Compiling model...
Model compiled.


1/1 [==============================] - 0s 126ms/step


1/1 [==============================] - 0s 297ms/step - loss: 0.0000e+00 - accuracy: 0.9512 - binary_crossentropy: 0.1555

fold_1: acc: 0.9597 loss: 0.1458

fold_2: acc: 0.8790 loss: 0.2557

fold_3: acc: 0.9268 loss: 0.2221

fold_4: acc: 0.9187 loss: 0.2841

fold_5: acc: 0.9512 loss: 0.1555

Average accuracy: 0.9271  Average loss: 0.2126


##Save the Model


In [ ]:
model.save('/content/drive/My Drive/ICR_Project/ICR_model.keras', save_format="keras")